In [22]:
%load_ext autoreload
%autoreload 2

import Unit_5 as u5
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Entering Data

In [23]:
node1 = u5.Point(0, 0, number=1)
node2 = u5.Point(0, 20 * 12, number=2)
node3 = u5.Point(20 * 12, 20 * 12, number=3)
node4 = u5.Point(12 * 20, 0, number=4)
E = 30 * 10 ** 6  #ib/in^2
A = 100  #in^2
I1 = 200 #in^4
I2 = 200 #in^4
I3 = 400 #in^4

In [30]:
# Elements
E1 = u5.Element(node1, node2, E, I1, A)
E2 = u5.Element(node2, node3, E, I2, A)
E3 = u5.Element(node3, node4, E, I3, A)

E1.make_element_K(node1, node2)
E2.make_element_K(node2, node3)
E3.make_element_K(node3, node4)

# print(E1.K)
# print(E2.K)
# print(E3.K)

#### Step 1: make total K as "K"
#### Step 2: make modified K as "K_modifide"

In [31]:
K = np.zeros(4 * 3 * 4 * 3)
K_N = u5.make_K_N(4 * 3)
K_modifide = np.array([])
# print(len(E1.K))
# print(len(E2.K))
K = u5.merge_K(K, K_N, E1)
K = u5.merge_K(K, K_N, E2)
K = u5.merge_K(K, K_N, E3)

# print(len(K))
# print(K)
new_K_N = u5.make_K_N(6, 2, 2)  # show where is the modified K
# print(new_K_N)
for p, v in np.ndenumerate(K_N):
    if v in new_K_N:
        K_modifide = np.append(K_modifide, K[p])
# print(new_K_N)
# print()
# print(K_N)
# print(K_modifide.reshape([3,3])/50000)
K_modifide = K_modifide.reshape([6, 6])  # shape the modified.
# print(K_modifide)

### F = K*D
#### find displacements as "displacements"

In [32]:
K_modifide_inv = np.linalg.inv(K_modifide)
F = np.array([[15000], [0], [0], [0], [0], [0]])  # Enter F matrix modified
displacements = np.dot(K_modifide_inv, F)
print(displacements.transpose())

[[ 1.57793834e+00  4.71362042e-04 -3.43363879e-03  1.57722413e+00
  -4.71362042e-04 -6.00145808e-03]]


### F = K*D
#### find Forces as "F_total"

In [33]:
displacements_total = np.zeros(4 * 3)  # total dimension
iteration = -1
temp = 0
# print(new_K_N)
new2_K_N = new_K_N.copy()
new2_K_N = new2_K_N[0:-1:6]  # n is the number of dimension in matrix
for i in range(len(new2_K_N)):
    new2_K_N[i] = new2_K_N[i][0:2]
# print(new2_K_N)
for p, v in np.ndenumerate(K_N):
    if temp == v[0:2]:
        continue
    temp = v[0:2]
    # print(v[0:2])
    # print(temp)
    # print("#")
    if v[0:2] in new2_K_N:
        iteration += 1
        # print(iter)
        displacements_total[p[0] // 12] = displacements[iteration]  # n is the dimension of main matrix... total K dimension
# print(displacements_total)
K = K.reshape([12, 12])
displacements_total = displacements_total.transpose()
# print(K.shape)
# print(displacements_total.shape)
F_total = np.dot(K, displacements_total)
print(F_total.transpose())

[-6.07240459e+03 -5.89202553e+03  8.14529521e+05  1.50000000e+04
  9.09494702e-13 -5.82076609e-11  0.00000000e+00 -9.09494702e-13
  2.91038305e-11 -8.92759541e+03  5.89202553e+03  1.37138435e+06]
